# load the data

In [6]:
import os
from pathlib import Path
import pandas as pd
import torch
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertTokenizer
from torch.utils.data import Dataset
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 加载数据集
current_path = os.path.realpath(__file__)
father_path = os.path.abspath(os.path.dirname(current_path)+os.path.sep+".")
data_path = Path(father_path, "fakenewskdd2020")
data = pd.read_csv(Path(data_path, 'train.csv'), sep='\t', encoding='utf-8')

# 去掉错误值
data = data.loc[data['label'].isin(['0', '1'])]
data = data.loc[data['text'].str.len() >=10]

# 处理缺失值
data.dropna(subset=['text', 'label'], inplace=True)

# 去掉重复数据
data.drop_duplicates(subset=["text"], inplace=True)
data.reset_index()

# 标签保证是int值
data['label'] = pd.to_numeric(data['label'], errors='coerce')
data['label'] = data['label'].astype(int)
                            
# 去除停用词
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
data['clean_text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

# 拆分训练数据
train_texts, test_texts, train_labels, test_labels = train_test_split(data['clean_text'], data['label'], test_size=0.2, random_state=42)

# 用BERT进行分词向量化
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 转换成输入token
max_length = 128
train_encodings = tokenizer(list(train_texts),     
                            add_special_tokens=True,  # 添加 [CLS] 和 [SEP] 标记
                            max_length=max_length,  # 填充和截断到最大长度
                            padding='max_length',  # 填充到最大长度
                            return_attention_mask=True,  # 返回注意力掩码
                            return_tensors='pt',  # 返回PyTorch张量
                            truncation=True)
test_encodings = tokenizer(list(test_texts),
                           add_special_tokens=True,  # 添加 [CLS] 和 [SEP] 标记
                           max_length=max_length,  # 填充和截断到最大长度
                           padding='max_length',  # 填充到最大长度
                           return_attention_mask=True,  # 返回注意力掩码
                           return_tensors='pt',  # 返回PyTorch张量
                           truncation=True)

# 对标签进行torch向量化转换
train_labels = torch.tensor(list(train_labels))
test_labels = torch.tensor(list(test_labels))


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# "创建一个PyTorch dataset 来使 BERT可以正常读取. 这个数据集包含tokenized和label."
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)


4. Build and Fine-tune BERT Model:


Next, we fine-tune BERT using the transformers library.

In [4]:
# 加载以BERT为底座的二分类模型
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# 设置训练参数
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=3,              # Number of epochs
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=50,                # Warmup steps for learning rate scheduler
    weight_decay=0.01,               # Weight decay
    evaluation_strategy="epoch",     # Evaluate after every epoch
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
)

# 初始化模型
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
# 开始训练
trainer.train()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: wandb version 0.18.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,0.750996,7.061400,2.832000
2,0.719700,0.695395,7.179800,2.786000
3,0.719700,0.668804,7.136800,2.802000


TrainOutput(global_step=15, training_loss=0.7108314196268718, metrics={'train_runtime': 369.5879, 'train_samples_per_second': 0.041, 'total_flos': 20180049960960.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': -111284224, 'init_mem_cpu_peaked_delta': 111284224, 'train_mem_cpu_alloc_delta': 5490667520, 'train_mem_cpu_peaked_delta': 144232448})

In [9]:
# 评估模型
results = trainer.evaluate()
print(f"Evaluation loss: {results['eval_loss']}")

# 验证集
predictions, labels, _ = trainer.predict(test_dataset)
predictions = torch.argmax(torch.tensor(predictions), axis=1)

# 计算性能指标
accuracy = accuracy_score(test_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(test_labels, predictions, average='binary')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Evaluation loss: 0.6688035726547241
Accuracy: 0.7
Precision: 0.6875
Recall: 0.9166666666666666
F1 Score: 0.7857142857142857
